In [1]:
!rm cz_2010.sqlite
import pandas as pd
import datetime
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime

In [2]:
df = pd.read_csv('data_in/LOS-ANGELES-DOWNTOWN_722874_CZ2010.CSV', low_memory=False)

In [3]:
df['date_time'] = df['Date (MM/DD/YYYY)'] + ' ' + df['Time (HH:MM)']

In [4]:
df.columns = df.columns.str.upper()

In [5]:
df.columns.tolist()

['DATE (MM/DD/YYYY)',
 'TIME (HH:MM)',
 'ETR (W/M^2)',
 'ETRN (W/M^2)',
 'GHI (W/M^2)',
 'GHI SOURCE',
 'GHI UNCERT (%)',
 'DNI (W/M^2)',
 'DNI SOURCE',
 'DNI UNCERT (%)',
 'DHI (W/M^2)',
 'DHI SOURCE',
 'DHI UNCERT (%)',
 'GH ILLUM (LX)',
 'GH ILLUM SOURCE',
 'GLOBAL ILLUM UNCERT (%)',
 'DN ILLUM (LX)',
 'DN ILLUM SOURCE',
 'DN ILLUM UNCERT (%)',
 'DH ILLUM (LX)',
 'DH ILLUM SOURCE',
 'DH ILLUM UNCERT (%)',
 'ZENITH LUM (CD/M^2)',
 'ZENITH LUM SOURCE',
 'ZENITH LUM UNCERT (%)',
 'TOTCLD (TENTHS)',
 'TOTCLD SOURCE',
 'TOTCLD UNCERT (CODE)',
 'OPQCLD (TENTHS)',
 'OPQCLD SOURCE',
 'OPQCLD UNCERT (CODE)',
 'DRY-BULB (C)',
 'DRY-BULB SOURCE',
 'DRY-BULB UNCERT (CODE)',
 'DEW-POINT (C)',
 'DEW-POINT SOURCE',
 'DEW-POINT UNCERT (CODE)',
 'RHUM (%)',
 'RHUM SOURCE',
 'RHUM UNCERT (CODE)',
 'PRESSURE (MBAR)',
 'PRESSURE SOURCE',
 'PRESSURE UNCERT (CODE)',
 'WDIR (DEGREES)',
 'WDIR SOURCE',
 'WDIR UNCERT (CODE)',
 'WSPD (M/S)',
 'WSPD SOURCE',
 'WSPD UNCERT (CODE)',
 'HVIS (M)',
 'HVIS SOURCE',

In [6]:
df_subset = df[['DATE (MM/DD/YYYY)', 'TIME (HH:MM)', 'DRY-BULB (C)']]

In [7]:
df_subset.columns = df_subset.columns.str.replace('(', '')
df_subset.columns = df_subset.columns.str.replace(')', '')
df_subset.columns = df_subset.columns.str.replace(' ', '_')
df_subset.columns = df_subset.columns.str.replace('/', '_')
df_subset.columns = df_subset.columns.str.replace('-', '_')
df_subset.columns = df_subset.columns.str.replace(':', '_')

In [8]:
df_subset.head()

,DATE_MM_DD_YYYY,TIME_HH_MM,DRY_BULB_C
0,2004-01-01,1:00,9.0
1,2004-01-01,2:00,7.8
2,2004-01-01,3:00,7.8
3,2004-01-01,4:00,7.2
4,2004-01-01,5:00,6.1


In [9]:
df_subset['TIME_DT_H'] = df_subset['TIME_HH_MM'].str.split(':').str[0]
df_subset['TIME_DT_HH'] = df_subset['TIME_DT_H'].apply(lambda x: x.zfill(2))
df_subset['TEMP_F'] = 'TEMP_F_' 
df_subset['TIME_DT_M'] = '00'
df_subset['DRY_BULB_F'] = (df_subset['DRY_BULB_C'] * (9/5)) + 32
df_subset['TYPE'] = 'CZ_2010'
df_subset['STATION'] = 'LOS-ANGELES-DOWNTOWN-USC'
df_subset['STATION_NBR'] = '722874'
df_subset['TIMESTAMP'] = df_subset[['TEMP_F', 'TIME_DT_HH', 'TIME_DT_M']].apply(lambda x: ''.join(x), axis=1)

/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [10]:
df_subset.head()

,DATE_MM_DD_YYYY,TIME_HH_MM,DRY_BULB_C,TIME_DT_H,TIME_DT_HH,TEMP_F,TIME_DT_M,DRY_BULB_F,TYPE,STATION,STATION_NBR,TIMESTAMP
0,2004-01-01,1:00,9.0,1,01,TEMP_F_,00,48.20,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,TEMP_F_0100
1,2004-01-01,2:00,7.8,2,02,TEMP_F_,00,46.04,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,TEMP_F_0200
2,2004-01-01,3:00,7.8,3,03,TEMP_F_,00,46.04,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,TEMP_F_0300
3,2004-01-01,4:00,7.2,4,04,TEMP_F_,00,44.96,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,TEMP_F_0400
4,2004-01-01,5:00,6.1,5,05,TEMP_F_,00,42.98,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874,TEMP_F_0500


In [11]:
FILTER = ['DATE_MM_DD_YYYY',
          'TIMESTAMP',
          'DRY_BULB_F',
          'TYPE',
          'STATION', 
          'STATION_NBR']

df = df_subset[FILTER]

In [12]:
df.head()

,DATE_MM_DD_YYYY,TIMESTAMP,DRY_BULB_F,TYPE,STATION,STATION_NBR
0,2004-01-01,TEMP_F_0100,48.20,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
1,2004-01-01,TEMP_F_0200,46.04,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
2,2004-01-01,TEMP_F_0300,46.04,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
3,2004-01-01,TEMP_F_0400,44.96,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874
4,2004-01-01,TEMP_F_0500,42.98,CZ_2010,LOS-ANGELES-DOWNTOWN-USC,722874


In [13]:
df_pivot = df.pivot_table(index=['DATE_MM_DD_YYYY', 'TYPE', 'STATION', 'STATION_NBR'], 
                          columns='TIMESTAMP',
                          values='DRY_BULB_F', 
                          aggfunc='sum')

In [14]:
#https://stackoverflow.com/questions/38951345/how-to-get-rid-of-multilevel-index-after-using-pivot-table-pandas
df_pivot = df_pivot.rename_axis(None, axis=1).reset_index() 

In [15]:
df_pivot.rename(columns={'DATE_MM_DD_YYYY': 'READ_DT'}, inplace=True)
df_pivot['READ_DT_MON'] = df_pivot['READ_DT'].str.split('-').str[1]
df_pivot['READ_DT_DAY'] = df_pivot['READ_DT'].str.split('-').str[2]
df_pivot['READ_DT_YEAR'] = df_pivot['READ_DT'].str.split('-').str[0]

In [16]:
df = df_pivot[[
         'READ_DT',
         'READ_DT_MON',
         'READ_DT_DAY',
         'READ_DT_YEAR',
         'TYPE',
         'STATION',
         'STATION_NBR',
         'TEMP_F_0000',
         'TEMP_F_0100',
         'TEMP_F_0200',
         'TEMP_F_0300',
         'TEMP_F_0400',
         'TEMP_F_0500',
         'TEMP_F_0600',
         'TEMP_F_0700',
         'TEMP_F_0800',
         'TEMP_F_0900',
         'TEMP_F_1000',
         'TEMP_F_1100',
         'TEMP_F_1200',
         'TEMP_F_1300',
         'TEMP_F_1400',
         'TEMP_F_1500',
         'TEMP_F_1600',
         'TEMP_F_1700',
         'TEMP_F_1800',
         'TEMP_F_1900',
         'TEMP_F_2000',
         'TEMP_F_2100',
         'TEMP_F_2200',
         'TEMP_F_2300']]

In [17]:
df.sort_values(by=['READ_DT_MON', 'READ_DT_DAY'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [18]:
df['READ_DT'] = pd.to_datetime(df['READ_DT'])

In [19]:
engine = create_engine("sqlite:///cz_2010.sqlite")

In [20]:
conn = engine.connect()
Base = declarative_base()

class Data(Base):
    __tablename__ = 'cz_2010_data'
    
    id = Column(Integer, primary_key=True)
    READ_DT = Column(DateTime)
    READ_DT_MON = Column(Integer)
    READ_DT_DAY = Column(Integer)
    READ_DT_YEAR = Column(Integer)
    TYPE = Column(String)
    STATION = Column(String)
    STATION_NBR = Column(Integer)
    TEMP_F_0000 = Column(Float)
    TEMP_F_0100 = Column(Float)
    TEMP_F_0200 = Column(Float)
    TEMP_F_0300 = Column(Float)
    TEMP_F_0400 = Column(Float)
    TEMP_F_0500 = Column(Float)
    TEMP_F_0600 = Column(Float)
    TEMP_F_0700 = Column(Float)
    TEMP_F_0800 = Column(Float)
    TEMP_F_0900 = Column(Float)
    TEMP_F_1000 = Column(Float)
    TEMP_F_1100 = Column(Float)
    TEMP_F_1200 = Column(Float)
    TEMP_F_1300 = Column(Float)
    TEMP_F_1400 = Column(Float)
    TEMP_F_1500 = Column(Float)
    TEMP_F_1600 = Column(Float)
    TEMP_F_1700 = Column(Float)
    TEMP_F_1800 = Column(Float)
    TEMP_F_1900 = Column(Float)
    TEMP_F_2000 = Column(Float)
    TEMP_F_2100 = Column(Float)
    TEMP_F_2200 = Column(Float)
    TEMP_F_2300 = Column(Float)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [21]:
Base.metadata.create_all(engine)

In [22]:
data_dict = df.to_dict(orient='records')

In [23]:
metadata = MetaData(bind = engine)
metadata.reflect()

In [24]:
table = sqlalchemy.Table('cz_2010_data', metadata, autoload=True)

In [25]:
conn.execute(table.delete())

In [26]:
conn.execute(table.insert(), data_dict)

In [30]:
conn.execute("SELECT SUM(TEMP_F_0000) \
              FROM cz_2010_data \
              WHERE READ_DT_MON=04").fetchall()

[(1667.4,)]

In [35]:
conn.execute("SELECT * \
              FROM cz_2010_data \
              WHERE READ_DT_MON=04").fetchall()[0:5]

[(91, '2001-04-01 00:00:00.000000', 4, 1, 2001, 'CZ_2010', 'LOS-ANGELES-DOWNTOWN-USC', 722874, 57.019999999999996, 62.42, 60.44, 59.18, 58.1, 57.019999999999996, 57.019999999999996, 57.019999999999996, 57.019999999999996, 55.94, 57.019999999999996, 59.0, 60.980000000000004, 62.06, 60.980000000000004, 64.94, 64.03999999999999, 60.980000000000004, 59.0, 59.0, 57.92, 57.92, 57.92, 57.92),
 (92, '2001-04-02 00:00:00.000000', 4, 2, 2001, 'CZ_2010', 'LOS-ANGELES-DOWNTOWN-USC', 722874, 57.019999999999996, 57.019999999999996, 57.019999999999996, 57.019999999999996, 55.94, 55.94, 55.94, 55.94, 57.019999999999996, 57.92, 60.08, 60.980000000000004, 62.96, 64.03999999999999, 64.94, 66.92, 64.03999999999999, 62.96, 62.06, 60.08, 57.92, 57.019999999999996, 55.94, 55.94),
 (93, '2001-04-03 00:00:00.000000', 4, 3, 2001, 'CZ_2010', 'LOS-ANGELES-DOWNTOWN-USC', 722874, 55.040000000000006, 57.019999999999996, 55.94, 55.94, 55.040000000000006, 53.96, 53.96, 53.96, 55.040000000000006, 55.94, 57.92, 59.0, 59